https://hub.docker.com/r/sath89/oracle-12c/

https://hub.docker.com/_/mysql/

https://stackoverflow.com/questions/8858008/how-to-move-a-file-in-python

In [1]:
# encoding=utf8
# import sys
# reload(sys)
# sys.setdefaultencoding('utf8')

import pandas as pd
import os
from time import time, ctime

import concurrent.futures

from lxml import etree
import json
import re


In [2]:
!ls|grep add

add_bidder
addbidderPics
addbidder_urls.csv
addboss.csv
addbossPics
addboss_urls.csv
add_owner
addtional
Parse_html_Stat-add1.ipynb
Parse_html_Stat-add2.ipynb
pic_down_func-add1.ipynb
pic_down_func-add2.ipynb


In [3]:
!pwd

/home/soyoung/face/Freelancer


In [4]:
# path='technosystem.html'

folder='add_bidder/'
cont = os.listdir(folder)
len(cont)

82195

In [5]:
def flatten_dict(data, layers=1, drop_deeper=False):

        for _ in range(layers):
            data = [(k, v) if not isinstance(v, dict) else [(k + '.' + k2, v2) for k2, v2 in v.items()] for k, v in
                    data.items()]
            data = [item for sublist in data for item in sublist if isinstance(sublist, list)] + [y for y in data if
                                                                                                  not isinstance(y,
                                                                                                                 list)]
            data = dict(data)

        if drop_deeper:
            data = {k: v for k, v in data.items() if not isinstance(v, dict) or isinstance(v, list)}

        return data

In [6]:
failed_names=[]

def get_columns(path):
    global failed_names
    get=[]
    
    with open(folder+path,'r') as fl:
        text=fl.read()
        tree=etree.HTML(text)
    try:
        script = tree.xpath('//script')[-1]
        data = re.search(r"__export\('profileServerData',(.*?)\n", script.text)        
        jsondata=data.group(1)[:-2]
        
        json_dict = json.loads(jsondata)
        flat = flatten_dict(json_dict, layers=1)
        names = flat.keys()
        return names
    except Exception as e:
        failed_names.append(path)
        return []

    

In [7]:
from collections import defaultdict
stat = defaultdict(int)

In [8]:
%%time
for i in cont:
    for k in get_columns(i):
        stat[k]+=1
len(stat),len(failed_names)

CPU times: user 6min 26s, sys: 5.62 s, total: 6min 32s
Wall time: 7min 24s


In [9]:
len(stat),len(failed_names)

(73, 17431)

In [10]:
col = pd.Series(stat.values(), index=stat.keys())
col.shape

(73,)

In [11]:
col.value_counts()

64764    72
52546     1
dtype: int64

In [12]:
col.sort_values(ascending=True).head()

hourlyRate               52546
about.support_status     64764
about.mobile_tracking    64764
about.timezone           64764
about.id                 64764
dtype: int64

In [13]:
print failed_names[:3]
len(failed_names)

['7323710.html', '6147909.html', '8871804.html']


17431

In [14]:
try:
    os.mkdir(folder+'NoJson')
except:
    print 'exists'

In [15]:
!ls {folder}'NoJson'|wc

      0       0       0


In [16]:
import shutil

In [17]:
%%time
# os.rename("path/to/current/file.foo", "path/to/new/desination/for/file.foo")
for i in failed_names:
    try:
        shutil.move(folder+i, folder+'NoJson/')
    except:
        pass


CPU times: user 192 ms, sys: 192 ms, total: 384 ms
Wall time: 387 ms


In [18]:
!ls {folder}'NoJson'|wc

  17431   17431  232625


In [19]:
!ls {folder}|wc

  64765   64765  860532


In [20]:
col.to_csv('addbidder.csv', header=['count'], index_label='name')

In [21]:
df1 = pd.read_csv('bidder1.csv').set_index('name')
df2 = pd.read_csv('boss1.csv').set_index('name')
df3 = pd.read_csv('addboss.csv').set_index('name')
df4 = pd.read_csv('addbidder.csv').set_index('name')

In [22]:
df1.shape, df2.shape, df3.shape, df4.shape

((73, 1), (73, 1), (73, 1), (73, 1))

In [27]:
(df1+df2+df3+df4).sort_values('count')

,count
name,
hourlyRate,103845
about.qualifications,150671
about.preferred_freelancer,150671
about.primary_currency,150671
about.primary_language,150671
about.profile_description,150671
about.public_name,150671
about.recommendations,150671
about.registration_date,150671


In [68]:
# group=[]
# def batch(batchfiles):
#     global group
#     group=[]
#     start=time()
#     with concurrent.futures.ThreadPoolExecutor(max_workers=8) as executor:
#         executor.map(get_single, batchfiles)
#     past='{:.2f}'.format(time()-start)
#     now=ctime()[4:]

#     print 'profiles ',len(batchfiles),'  used ',past,'s', now
#     return group


In [74]:
len(batch(cont[:2]))

profiles  2   used  0.02 s Sep 28 01:07:10 2017


2

In [84]:
len(batch(cont[:10])[6])

'NoneType' object has no attribute 'group'
'NoneType' object has no attribute 'group'
profiles  10   used  0.06 s Sep 28 01:08:03 2017


168